# Description
This notebook has an interactive widget of the asymmetric logistic function, helping to build graphical intuition.

At the same time it creates the corresponding plot for the manuscript.

In [1]:
import numpy
from matplotlib import pyplot
import ipywidgets

import calibr8
import plotting

### The Generalized Logistic Function
Real-world calibration curves rarely follow linear dose/response relationships, but often exhibit lower & upper saturations.
These relationships can be described with S-shaped functions.
The logistic function (the _sigmoid curve_ is a special case of it) is often well suited for real-world calibration curves.

Several different flavors of S-shaped curves are available:
+ [sigmoid](https://en.wikipedia.org/wiki/Sigmoid_function) (1 parameter: variable slope)
+ [logistic](https://en.wikipedia.org/wiki/Logistic_function) (3 parameters: variable upper limit, variable x-value of the inflection point, variable slope)
+ [generalized logistic](https://en.wikipedia.org/wiki/Generalised_logistic_function) (5 parameters: variable limits, variable inflection point, variable slope, variable symmetry)

Here, we're going to use the generalized logistic, because it is most useful. However, we use a different parametrization that is derived in the notebook `Background_AsymmetricLogistic`.

$$f(x)= L_L \cdot (L_U - L_L) \cdot (e^{(\frac{S}{L_U - L_L} (I_{x} - x) + c (e^{c} + 1)^{-(e^{c} + 1) e^{- c}}) (e^{c} + 1)^{(e^{c} + 1) e^{- c}}} + 1)^{- e^{- c}} $$

The following table shows the bounds & interpretation of its 5 paramters:

| parameter     | interpretation
| ----- | -- |
| $L_L \in \mathcal{R} $ | lower asymptote |
| $L_U \in \mathcal{R} $ | upper asymptote |
| $I_x \in \mathcal{R} $ | x-value of the inflection point |
| $S \in \mathcal{R} $ | slope at the inflection point $I_x$ |
| $c \in \mathcal{R} $ | moves the inflection point between $L_L$ ($c < 0$) and $L_U$ ($0 < c$) limit |

### Example
Let's say we have an assay with noticeable lower- and upper satuation limits, as well as some asymmetry.

To visualize the meaning of the parameters, we can draw an interactive plot of the `calibr8.asymmetric_logistic` function:

In [2]:
help(calibr8.asymmetric_logistic)

Help on function asymmetric_logistic in module calibr8.core:

asymmetric_logistic(x, theta)
    5-parameter asymmetric logistic model.
    
    Parameters
    ----------
    x : array-like
        independent variable
    theta : array-like
        parameters of the logistic model
            L_L: lower asymptote
            L_U: upper asymptote
            I_x: x-value at inflection point
            S: slope at the inflection point
            c: symmetry parameter (0 is symmetric)
    
    Returns
    -------
    y : array-like
        dependent variable



In [3]:
def tangent(I_x, I_y, S):
    """Get x,y to plot a line with slope S around the coordinate <I_x,I_y>."""
    x = numpy.linspace(I_x - 1, I_x + 1, 2)
    y = -S * I_x + I_y + S * x
    return x, y


def plot_logistic(L_L=-0.5, L_U=1.5, I_x=-1, S=1.3, c=-1.5):
    theta = (L_L, L_U, I_x, S, c)
    X_MIN, X_MAX = -4, 4
    X = numpy.linspace(X_MIN, X_MAX, 200)

    # get key properties to visualize
    I_y = calibr8.asymmetric_logistic(I_x, theta)

    fig, ax = pyplot.subplots(figsize=(8, 3), dpi=300)
    # draw with different c parameter
    for c, col in [
        (1, "gray"),
        (0, "gray"),
        (theta[-1], "black"),
    ]:
        t = theta[:-1] + (c,)
        I_y = calibr8.asymmetric_logistic(I_x, t)
        ax.plot(X, calibr8.asymmetric_logistic(X, t), color=col)
        ax.annotate(
            xy=(I_x, I_y),
            xytext=(I_x - 1, I_y + 0.5),
            text="$c={" + str(c) + "}$",
            fontsize=6,
            color=col,
            horizontalalignment="right",
            verticalalignment="baseline",
            arrowprops=dict(arrowstyle="-|>", facecolor=col, edgecolor=col),
        )

    bbox_settings = dict(facecolor="white", edgecolor="white")
    tang_x, tang_y = tangent(I_x, I_y, S)
    ax.plot(tang_x, tang_y, color="orange", label="tangent in inflection point")
    ax.text(
        x=tang_x[-1] + 0.05,
        horizontalalignment="left",
        y=tang_y[-1] + 0.05,
        verticalalignment="bottom",
        s="tangent at inflection point",
        fontsize=10,
        color="orange",
    )

    ax.axvline(I_x, linestyle=":", color="orange", label="$I_x$")
    ax.text(
        x=I_x,
        horizontalalignment="center",
        y=I_y - (L_U - L_L) / 4,
        s="$I_x$",
        color="orange",
        fontsize=10,
    ).set_bbox(bbox_settings)

    ax.axhline(I_y, linestyle=":", color="orange", label="$I_y$")
    ax.text(
        x=I_x + (L_U - L_L) / 4,
        y=I_y,
        verticalalignment="center",
        s="$I_y$",
        color="orange",
        fontsize=10,
    ).set_bbox(bbox_settings)

    ax.axhline(L_U, linestyle="--", label="$L_U$", color="blue")
    ax.text(
        x=3, y=L_U, verticalalignment="center", s="$L_U$", fontsize=10, color="blue"
    ).set_bbox(bbox_settings)

    ax.axhline(L_L, linestyle="--", label="$L_L$", color="blue")
    ax.text(
        x=-3, y=L_L, verticalalignment="center", s="$L_L$", fontsize=10, color="blue"
    ).set_bbox(bbox_settings)

    ax.set_xlim(X_MIN, X_MAX)
    ax.set_ylim(L_L - 0.5, L_U + 0.5)
    plotting.savefig(fig, "3.2.1 Asymmetric Logistic")
    pyplot.show()


ipywidgets.interact(
    plot_logistic, L_L=(-5.0, 0), L_U=(0.0, 5), I_x=(-5.0, 5), S=(-2.0, 3), c=(-2.0, 2)
);

interactive(children=(FloatSlider(value=-0.5, description='L_L', max=0.0, min=-5.0), FloatSlider(value=1.5, de…

### Summary
+ The generalized logistic function can describe relationships with lower & upper satuation
+ Even in cases of "good linear relationships", the GLF is applicable
+ The original parameterization is not very intuitive, but was reparameterized such that 4 of 5 parameters are interpretable

In [4]:
%load_ext watermark
%watermark

Last updated: 2021-12-14T20:52:11.395681+01:00

Python implementation: CPython
Python version       : 3.7.9
IPython version      : 7.24.1

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 158 Stepping 10, GenuineIntel
CPU cores   : 6
Architecture: 64bit

